ML 2 — Entraînement d’un modèle prédictif sur le diabète

🩺 1. Chargement et préparation des données

In [10]:
import pandas as pd
import numpy as np
import joblib

# Pour la modélisation et la préparation des données
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

# Pour l'évaluation des performances
from sklearn.metrics import classification_report, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
plt.style.use("fivethirtyeight")

⚙️ 2. Préparation du jeu d'entraînement et de test

In [11]:
print("--- 1. Configuration des chemins de fichiers ---")
OUTPUT_PREDICTIONS_FILE = 'predictions_test.csv'
MODEL_ARTIFACT = 'modele_diabete_XX.pkl'  # <-- Remplacer XX par tes initiales

print("\n--- 2. Chargement des ensembles Train et Test ---")

try:
    df_train = pd.read_csv('data/diabetes_clean.csv')
    df_test_raw = pd.read_csv('data/test_without_class.csv')
except FileNotFoundError as e:
    print(f"ERREUR : Fichier non trouvé - {e.filename}")
    exit()

X_train_full = df_train.drop('class', axis=1)
y_train_full = df_train['class']

test_ids = df_test_raw['ID']
X_test = df_test_raw.drop('ID', axis=1).copy()

X_test.columns = [col.lower().replace(' ', '_') for col in X_test.columns]

for col in X_test.columns:
    if X_test[col].dtype == 'object':
        if col == 'gender':
            mapping = {'Female': 0, 'Male': 1}
        else:
            mapping = {'No': 0, 'Yes': 1}
        X_test[col] = X_test[col].map(mapping)
        X_test[col] = X_test[col].fillna(0).astype(int)

assert X_test.select_dtypes(include='object').empty, "Certaines colonnes du test ne sont pas numériques."
print("Encodage du test terminé ✅")

--- 1. Configuration des chemins de fichiers ---

--- 2. Chargement des ensembles Train et Test ---
Encodage du test terminé ✅


# ============================================================
# 3. PIPELINE DE PRÉTRAITEMENT ET ENTRAÎNEMENT FINAL
# ============================================================

In [12]:
print("\n--- 3. Entraînement du modèle optimisé ---")

numerical_features = X_train_full.select_dtypes(include=['int64', 'float64']).columns
preprocessor = ColumnTransformer(
    transformers=[('scaler', StandardScaler(), numerical_features)],
    remainder='passthrough'
)

# 🧠 RandomForest surentraîné (mode performance maximale)
best_model = RandomForestClassifier(
    n_estimators=2000,       # beaucoup d'arbres
    max_depth=None,          # pas de limite de profondeur
    min_samples_split=2,
    min_samples_leaf=1,
    bootstrap=False,         # désactive le bootstrap → surapprentissage complet
    random_state=42
)

final_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', best_model)
])

final_pipeline.fit(X_train_full, y_train_full)
print("✅ Entraînement terminé.")



--- 3. Entraînement du modèle optimisé ---
✅ Entraînement terminé.


# ============================================================
# 4. ÉVALUATION DU SCORE D'ENTRAÎNEMENT
# ============================================================

In [13]:
train_score = final_pipeline.score(X_train_full, y_train_full)
print(f"\n🔥 Score sur l'ensemble d'entraînement : {train_score:.3f}")

if train_score == 1.0:
    print("🎯 Parfait ! Score de 1.000 atteint sur le jeu d'entraînement.")
else:
    print("⚠️ Score inférieur à 1.000. Peut-être que certaines valeurs sont encore imprévisibles.")


🔥 Score sur l'ensemble d'entraînement : 1.000
🎯 Parfait ! Score de 1.000 atteint sur le jeu d'entraînement.


# ============================================================
# 5. IMPORTANCE DES VARIABLES
# ============================================================

In [14]:
importances = final_pipeline.named_steps['classifier'].feature_importances_
cols = X_train_full.columns
feat_importance = pd.Series(importances, index=cols).sort_values(ascending=False)
print("\nTop 10 des features les plus importantes :")
print(feat_importance.head(10))


Top 10 des features les plus importantes :
polyuria              0.210236
polydipsia            0.182497
gender                0.116260
age                   0.099688
sudden_weight_loss    0.059015
alopecia              0.042205
partial_paresis       0.041916
irritability          0.036075
delayed_healing       0.035971
itching               0.033657
dtype: float64


# ============================================================
# 6. PREDICTIONS ET EXPORT
# ============================================================

In [15]:

predictions_binary = final_pipeline.predict(X_test)

submission_df = pd.DataFrame({
    'ID': test_ids,
    'class': predictions_binary
})

submission_df.to_csv(OUTPUT_PREDICTIONS_FILE, index=False)
print(f"\n✅ Fichier de soumission créé : '{OUTPUT_PREDICTIONS_FILE}'")
print(submission_df.head())

joblib.dump(final_pipeline, MODEL_ARTIFACT)
print(f"✅ Modèle sauvegardé : '{MODEL_ARTIFACT}'")

print("\n--- FIN DU SCRIPT ---")


✅ Fichier de soumission créé : 'predictions_test.csv'
    ID  class
0  417      0
1  418      1
2  419      1
3  420      0
4  421      0
✅ Modèle sauvegardé : 'modele_diabete_XX.pkl'

--- FIN DU SCRIPT ---
